# MCM2017 B

## Problem B: Merge After Toll
Multi-lane divided limited-access toll highways use “ramp tolls” and “barrier tolls” to collect tolls from motorists. A ramp toll is a collection mechanism at an entrance or exit ramp to the highway and these do not concern us here. A barrier toll is a row of tollbooths placed across the highway, perpendicular to the direction of traffic flow. There are usually (always) more tollbooths than there are incoming lanes of traffic (see former 2005 MCM Problem B). So when exiting the tollbooths in a barrier toll, vehicles must “fan in” from the larger number of tollbooth egress lanes to the smaller number of regular travel lanes. A toll plaza is the area of the highway needed to facilitate the barrier toll, consisting of the fan-out area before the barrier toll, the toll barrier itself, and the fan-in area after the toll barrier. For example, a three-lane highway (one direction) may use 8 tollbooths in a barrier toll. After paying toll, the vehicles continue on their journey on a highway having the same number of lanes as had entered the toll plaza (three, in this example). 

Consider a toll highway having L lanes of travel in each direction and a barrier toll containing B tollbooths (B > L) in each direction. Determine the shape, size, and merging pattern of the area following the toll barrier in which vehicles fan in from B tollbooth egress lanes down to L lanes of traffic. Important considerations to incorporate in your model include accident prevention, throughput (number of vehicles per hour passing the point where the end of the plaza joins the L outgoing traffic lanes), and cost (land and road construction are expensive). In particular, this problem does not ask for merely a performance analysis of any particular toll plaza design that may already be implemented. The point is to determine if there are better solutions (shape, size, and merging pattern) than any in common use. 

Determine the performance of your solution in light and heavy traffic. How does your solution change as more autonomous (self-driving) vehicles are added to the traffic mix? How is your solution affected by the proportions of conventional (human-staffed) tollbooths, exact-change (automated) tollbooths, and electronic toll collection booths (such as electronic toll collection via a transponder in the vehicle)?


### Link 2005 MCM Problem B （Not Problem-B 2017）
PROBLEM B: Tollbooths

Heavily-traveled toll roads such as the Garden State Parkway , Interstate 95, and so forth, are multi-lane divided highways that are interrupted at intervals by toll plazas. Because collecting tolls is usually unpopular, it is desirable to minimize motorist annoyance by limiting the amount of traffic disruption caused by the toll plazas. Commonly, a much larger number of tollbooths is provided than the number of travel lanes entering the toll plaza. Upon entering the toll plaza, the flow of vehicles fans out to the larger number of tollbooths, and when leaving the toll plaza, the flow of vehicles is required to squeeze back down to a number of travel lanes equal to the number of travel lanes before the toll plaza. Consequently, when traffic is heavy, congestion increases upon departure from the toll plaza. When traffic is very heavy, congestion also builds at the entry to the toll plaza because of the time required for each vehicle to pay the toll.

Make a model to help you determine the optimal number of tollbooths to deploy in a barrier-toll plaza. Explicitly consider the scenario where there is exactly one tollbooth per incoming travel lane. Under what conditions is this more or less effective than the current practice? Note that the definition of “optimal” is up to you to determine.

## 探索([数据](http://metrocosm.com/get-the-data/#interstatetraffic))
### 1. Traffic counting stations:  
csv: http://metrocosm.com/wp-content/uploads/2015/03/sta2015.csv

In [1]:
import pandas as pd
import gmaps
from key import GOOGLE_KEY
gmaps.configure(api_key=GOOGLE_KEY)

#### 导入数据

In [6]:
df_tcs = pd.read_csv('http://metrocosm.com/wp-content/uploads/2015/03/sta2015.csv', error_bad_lines=False);

b'Skipping line 2014: expected 44 fields, saw 45\nSkipping line 2015: expected 44 fields, saw 45\nSkipping line 2016: expected 44 fields, saw 45\nSkipping line 2017: expected 44 fields, saw 45\nSkipping line 2018: expected 44 fields, saw 45\nSkipping line 2019: expected 44 fields, saw 45\nSkipping line 2028: expected 44 fields, saw 45\nSkipping line 2029: expected 44 fields, saw 45\nSkipping line 2030: expected 44 fields, saw 45\nSkipping line 2031: expected 44 fields, saw 45\nSkipping line 2032: expected 44 fields, saw 45\nSkipping line 2033: expected 44 fields, saw 45\nSkipping line 2060: expected 44 fields, saw 45\nSkipping line 2061: expected 44 fields, saw 45\nSkipping line 2062: expected 44 fields, saw 45\nSkipping line 2063: expected 44 fields, saw 45\nSkipping line 2064: expected 44 fields, saw 45\nSkipping line 2065: expected 44 fields, saw 45\nSkipping line 2066: expected 44 fields, saw 45\nSkipping line 2067: expected 44 fields, saw 45\nSkipping line 2068: expected 44 fields

In [9]:
df_tcs.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21571 entries, 0 to 21570
Data columns (total 44 columns):
Year_Record             21571 non-null object
State_Code              21570 non-null object
Station_Id              21570 non-null object
Travel_Dir              21570 non-null object
Travel_Lane             21570 non-null object
F_System                21570 non-null object
Num_Lanes               21488 non-null object
Sample_Type_Volume      21570 non-null object
Num_Lanes_Volume        21570 non-null object
Method_Volume           20926 non-null object
Sample_Type_Class       21570 non-null object
Num_Lanes_Class         21238 non-null object
Method_Class            14841 non-null object
Algorithm_Volume        21570 non-null object
Num_Classes             21168 non-null object
Sample_Type_Truck       21570 non-null object
Num_Lanes_Truck         21242 non-null object
Method_Truck            9621 non-null object
Calibration             21570 non-null object
Data_Retrieval    

#### 清理数据

In [41]:
df_tcs.drop(df_tcs[df_tcs.Station_Id.str.len() != len('000002')].index, inplace=True)  # 去掉id不正确的行
df_tcs['Station_Name'] = df_tcs.State_Code + '_' + df_tcs.Station_Id  # 把state_code与station_id拼接起来

df_tcs_sn_vcs = df_tcs.Station_Name.value_counts()  # 暂存station_name的value counts
df_tcs_sn_vcs[df_tcs_sn_vcs == 4].head(5)

56_000046    4
39_000745    4
16_000275    4
20_AY7VU2    4
13_000298    4
Name: Station_Name, dtype: int64

#### !!发现
*Travel_Dir* $\times$ *Travel_Lane* $=$ *Station_Name*

In [12]:
def get_sn_td_tl(station_name):
    sn = df_tcs[df_tcs.Station_Name == station_name].shape[0]
    td = df_tcs[df_tcs.Station_Name == station_name].Travel_Dir.value_counts().shape[0]
    tl = df_tcs[df_tcs.Station_Name == station_name].Travel_Lane.value_counts().shape[0]
    return (sn, td, tl)
get_sn_td_tl('13_000168')

(2, 2, 1)

#### 清理路名NaN

In [59]:
df_tcs.drop(df_tcs[df_tcs.Station_Location.isna()].index, inplace=True)

#### 看其中一个station

In [67]:
df_one_station = df_tcs[df_tcs.Station_Name == '35_P00200']
df_one_station.iloc[0]

Year_Record                                                          2015
State_Code                                                             35
Station_Id                                                         P00200
Travel_Dir                                                              1
Travel_Lane                                                             1
F_System                                                               11
Num_Lanes                                                               2
Sample_Type_Volume                                                      T
Num_Lanes_Volume                                                        2
Method_Volume                                                           3
Sample_Type_Class                                                        
Num_Lanes_Class                                                         0
Method_Class                                                          NaN
Algorithm_Volume                      

#### 使用Google Map 探索
##### 提取经纬度数据

In [15]:
df_station_position = df_tcs[['Latitude', 'Longitude']].copy()
df_station_position[df_station_position.Latitude.isna()]

,Latitude,Longitude
438,NaN,NaN


##### 清理不正确的经纬度

In [17]:
df_clean_position = df_station_position.drop(df_station_position[df_station_position.Latitude.isna()].index).copy()
df_clean_position.drop(df_clean_position[df_clean_position.Latitude == '0'].index, inplace=True)

##### 把经纬度数据转化为正确的经纬度格式

In [19]:
import math

def to_right_titude(titude):
    num = len(titude)
    if titude.startswith('1'):
        base = 10 ** (num-3)
    else:
        base = 10 ** (num-2)
    assert base > 0
    result = int(titude)
    return -result/base

def to_left_titude(titude):
    num = len(titude)
    base = 10 ** (num-2)
    assert base > 0
    result = int(titude)
    return result/base


##### 执行转化

In [20]:
df_position = df_clean_position.copy()
df_position.Latitude = df_clean_position.Latitude.apply(to_left_titude)
df_position.Longitude = df_clean_position.Longitude.apply(to_right_titude)

##### 丢弃不能用的经纬度数据

In [26]:
indexss = df_position[df_position.Latitude > 90].index
df_position.drop(indexss, inplace=True)
df_position.shape

(21267, 2)

#### 搜索收费站，然后找路
先去Google Map 搜索"toll plazas"，找到一些收费站及其所在的路名，然后再找Station

In [103]:
df_tcs[df_tcs.Station_Location.str.contains('I-95')]

,Year_Record,State_Code,Station_Id,Travel_Dir,Travel_Lane,F_System,Num_Lanes,Sample_Type_Volume,Num_Lanes_Volume,Method_Volume,...,Posted_Route_Signing,Posted_Signed_Route,Con_Route_Signing,Con_Signed_Route,Station_Location,Class_Change_Date,Class14_distribution,Class15_distribution,New_FSystem,Station_Name
4784,2015,13,000018,1,1,11,3,T,3,3,...,1,00000095,3,00000405,I-95/SR405 2 MI N OF SR21 IN SAVANNH MP 110 & ...,NaN,0,0,0,13_000018
4785,2015,13,000018,1,2,11,3,T,3,3,...,1,00000095,3,00000405,I-95/SR405 2 MI N OF SR21 IN SAVANNH MP 110 & ...,NaN,0,0,0,13_000018
4786,2015,13,000018,1,3,11,3,T,3,3,...,1,00000095,3,00000405,I-95/SR405 2 MI N OF SR21 IN SAVANNH MP 110 & ...,NaN,0,0,0,13_000018
4787,2015,13,000018,5,1,11,3,T,3,3,...,1,00000095,3,00000405,I-95/SR405 2 MI N OF SR21 IN SAVANNH MP 110 & ...,NaN,0,0,0,13_000018
4788,2015,13,000018,5,2,11,3,T,3,3,...,1,00000095,3,00000405,I-95/SR405 2 MI N OF SR21 IN SAVANNH MP 110 & ...,NaN,0,0,0,13_000018
4789,2015,13,000018,5,3,11,3,T,3,3,...,1,00000095,3,00000405,I-95/SR405 2 MI N OF SR21 IN SAVANNH MP 110 & ...,NaN,0,0,0,13_000018
4874,2015,13,000161,1,1,11,4,T,4,3,...,1,00000095,3,00000405,I-95/SR405 BTWN SR27 & GOLDEN ISLES PKWY SR 25...,NaN,0,0,0,13_000161
4875,2015,13,000161,1,2,11,4,T,4,3,...,1,00000095,3,00000405,I-95/SR405 BTWN SR27 & GOLDEN ISLES PKWY SR 25...,NaN,0,0,0,13_000161
4876,2015,13,000161,1,3,11,4,T,4,3,...,1,00000095,3,00000405,I-95/SR405 BTWN SR27 & GOLDEN ISLES PKWY SR 25...,NaN,0,0,0,13_000161
4877,2015,13,000161,1,4,11,4,T,4,3,...,1,00000095,3,00000405,I-95/SR405 BTWN SR27 & GOLDEN ISLES PKWY SR 25...,NaN,0,0,0,13_000161


In [104]:
df_road = df_tcs[df_tcs.Station_Location.str.contains('I-95')].copy()

#### 用Google Map找出离收费站最近的station

In [105]:
toll_plaza = pd.DataFrame([[39.266496, -76.562375]])
toll_plaza_layer = gmaps.symbol_layer(
    toll_plaza, fill_color='green', stroke_color='green', scale=2
)

df_test = df_position.loc[df_road.index].copy()
station_layer = gmaps.symbol_layer(
    df_test, fill_color='red', stroke_color='red', scale=2
)

#### 显示地图

In [107]:
fig = gmaps.figure()
fig.add_layer(station_layer)
fig.add_layer(toll_plaza_layer)
fig

Figure()